# Purpose


This notebook demonstrates the data pipeline from raw tables to analytical datasets. At the end of this activity, train & test data sets are created from raw data.



## Imports

In [14]:
from pprint import pprint
import os
import os.path as op
import shutil

# standard third party imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.mode.use_inf_as_na = True


In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
# standard code-template imports
from ta_lib.core.api import (
    create_context, get_dataframe, get_feature_names_from_column_transformer, get_package_path,
    display_as_tabs, string_cleaning, merge_info, initialize_environment,
    list_datasets, load_dataset, save_dataset
)
import ta_lib.eda.api as eda

In [17]:
initialize_environment(debug=False, hide_warnings=True)

## Utility functions

# 1. Initialization

In [18]:
config_path = op.join('conf', 'config.yml')
context = create_context(config_path)
pprint(list_datasets(context))

housing_df = load_dataset(context, 'raw/housing')

['/raw/housing',
 '/cleaned/housing',
 '/processed/housing',
 '/train/housing/features',
 '/train/housing/target',
 '/test/housing/features',
 '/test/housing/target',
 '/score/housing/output']


# 2. Data cleaning and consolidation

**<u>NOTES</u>**

The focus here is to create a cleaned dataset that is appropriate for solving the DS problem at hand from the raw data.

**1. Do**
* clean dataframe column names
* ensure dtypes are set properly
* join with other tables etc to create features
* transform, if appropriate, datetime like columns to generate additional features (weekday etc)
* transform, if appropriate, string columns to generate additional features
* discard cols that are not useful for training the model (IDs, constant cols, duplicate cols etc)
* additional features generated from existing columns


**2. Don't**
* handle missing values or outliers here. mark them and leave them for processing downstream.


## 2.1 Clean table 

### Housing Table

From data discovery, we know the following

* ensure all `invalid` entries are mapped to np.NaN
* This will go into production code


In [19]:
housing_df_clean = (
    housing_df
    # while iterating on testing, it's good to copy the dataset(or a subset)
    # as the following steps will mutate the input dataframe. The copy should be
    # removed in the production code to avoid introducing perf. bottlenecks.
    .copy()
    
    .passthrough()
    
    .replace({'': np.NaN})
    
)
housing_df_clean.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


### NOTE

It's always a good idea to save cleaned tabular data using a storage format that supports the following 

1. preserves the type information
2. language agnostic storage format
3. Supports compression
4. Supports customizing storage to optimize different data access patterns

For larger datasets, the last two points become crucial.

`Parquet` is one such file format that is very popular for storing tabular data. It has some nice properties:
- Similar to pickles & RDS datasets, but compatible with all languages
- Preserves the datatypes
- Compresses the data and reduces the filesize
- Good library support in Python and other languages
- As a columnar storage we can efficiently read fewer columns
- It also supports chunking data by groups of columns (for instance, by dates or a particular value of a key column) that makes loading subsets of the data fast.

In [20]:
save_dataset(context, housing_df_clean, 'cleaned/housing')

## 2.2 Create consolidated features table

Here we take the cleaned dataframes and added some feature to form the consolidated table.



In [21]:
# Relevant Features of a house.
housing_df["rooms_per_household"] = housing_df["total_rooms"]/housing_df["households"]
housing_df["bedrooms_per_room"] = housing_df["total_bedrooms"]/housing_df["total_rooms"]
housing_df["population_per_household"]=housing_df["population"]/housing_df["households"]

## 2.3 Business intuition features

This section can go into production code if these features are used in final model

#### Extra Relevant Features of a house
 - features that tells more about a house like rooms per house, bedrooms per house etc.

In [22]:
# create a sample dataframe with minimal processing

housing_df_processed = (
  housing_df
    
    #tweak to test pipeline quickly or profile performance
    .sample(frac=1, replace=False)
    
    # any additional processing/cleaning
)

# Any verifications on the data
from ta_lib.eda.api import get_variable_summary
display_as_tabs([
    ("Summary", f"Length: {len(housing_df_processed)}, Columns: {len(housing_df_processed.columns)}"),
    ("Variable summary", get_variable_summary(housing_df_processed)),
    ("head", housing_df.head(5).T),
    ("tail", housing_df.tail(5).T),
])

Tabs
    [0] Markdown(str, name='Summary')
    [1] DataFrame(name='Variable summary', value=                          ...)
    [2] DataFrame(name='head', value=              ...)
    [3] DataFrame(name='tail', value=              ...)

In [23]:
save_dataset(context, housing_df_processed, 'processed/housing')

# 3. Generate Train, Validation and Test datasets



- We split the data into train, test (optionally, also a validation dataset)
- In this example, we are binning the target into 10 quantiles and then use a Stratified Shuffle to split the data.
- See sklearn documentation on the various available splitters
- https://scikit-learn.org/stable/modules/classes.html#splitter-classes
- This will go into production code (training only)

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit
from ta_lib.core.api import custom_train_test_split  # helper function to customize splitting
from scripts import *

splitter = StratifiedShuffleSplit(n_splits=1,test_size=0.2, random_state=42)
housing_df_train, housing_df_test = custom_train_test_split(housing_df_processed, splitter, by=binned_selling_price)

In [25]:
housing_df_train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity,rooms_per_household,bedrooms_per_room,population_per_household
11318,-117.99,33.77,15.0,2081.0,531.0,1617.0,561.0,3.4955,160900.0,<1H OCEAN,3.709447,0.255166,2.882353
9781,-121.13,36.20,16.0,1868.0,443.0,1323.0,436.0,2.9559,163200.0,<1H OCEAN,4.284404,0.237152,3.034404
4321,-118.35,34.08,52.0,1003.0,200.0,514.0,204.0,3.8472,395700.0,<1H OCEAN,4.916667,0.199402,2.519608
7838,-118.17,33.90,12.0,3653.0,993.0,3215.0,854.0,2.8681,114200.0,<1H OCEAN,4.277518,0.271831,3.764637
14311,-117.14,32.71,39.0,1647.0,478.0,2176.0,479.0,1.7642,82900.0,NEAR OCEAN,3.438413,0.290225,4.542797
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15382,-117.14,33.39,17.0,2889.0,587.0,1931.0,510.0,3.8547,208300.0,<1H OCEAN,5.664706,0.203184,3.786275
4052,-118.43,34.15,42.0,1293.0,214.0,459.0,217.0,7.6720,467600.0,<1H OCEAN,5.958525,0.165507,2.115207
18505,-121.98,37.14,37.0,74.0,19.0,63.0,17.0,9.5908,350000.0,<1H OCEAN,4.352941,0.256757,3.705882
17814,-121.82,37.42,13.0,3752.0,572.0,1581.0,526.0,6.1091,329400.0,<1H OCEAN,7.133080,0.152452,3.005703


In [12]:
target_col = "median_house_value"

train_X, train_y = (
    housing_df_train
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, train_X, 'train/housing/features')
save_dataset(context, train_y, 'train/housing/target')


test_X, test_y = (
    housing_df_test
    
    # split the dataset to train and test
    .get_features_targets(target_column_names=target_col)
)
save_dataset(context, test_X, 'test/housing/features')
save_dataset(context, test_y, 'test/housing/target')

In [13]:
test_X.isnull().sum()

longitude                    0
latitude                     0
housing_median_age           0
total_rooms                  0
total_bedrooms              39
population                   0
households                   0
median_income                0
ocean_proximity              0
rooms_per_household          0
bedrooms_per_room           39
population_per_household     0
dtype: int64